This notebook uses a BiLSTM+Attention model to predict binary sentiment.<br>
The data input pipeline reads from a generator based on a pandas dataframe.

In [1]:
import os, math, pickle
from gensim.models import KeyedVectors
from shutil import rmtree
import tensorflow as tf
import numpy as np
import pandas as pd
seed = 123
from sklearn.metrics import f1_score, accuracy_score, classification_report

tf.logging.set_verbosity(tf.logging.INFO)
print(tf.__version__)

C:\Users\Lance\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


1.12.0


In [2]:
batch_size = 512
max_len = 50
n_instances = 26790665
n_classes = 2
steps = math.ceil(n_instances/batch_size)
print("Total steps:", steps)

Total steps: 52326


In [3]:
# read zero-padded sequences
feature_names = [str(col) for col in range(max_len)]
all_names = feature_names.copy()
all_names.append("label")
df_data = pd.read_csv(r"D:\stocktwits_text\2018 binary\seq.csv", 
                     names=all_names)
print(df_data["label"].value_counts())
df_data.head()

1    22163105
0     4627560
Name: label, dtype: int64


,0,1,2,3,4,5,6,7,8,9,...,41,42,43,44,45,46,47,48,49,label
0,3,1268,41,10,183,74,20,4,98,27,...,0,0,0,0,0,0,0,0,0,1
1,1,37,21,80,86,17,4474,801,109,6,...,0,0,0,0,0,0,0,0,0,1
2,1,9,50,99,671,1088,1239,28432,1821,12,...,0,0,0,0,0,0,0,0,0,1
3,1,3,13796,1187,2,9,134,15,246,26,...,0,0,0,0,0,0,0,0,0,0
4,1,44,250,64,4,3105,82,1777,0,0,...,0,0,0,0,0,0,0,0,0,1


In [4]:
# train-test split
test_size = len(df_data)//10
print(test_size)
test_indices = set(np.random.choice(df_data.index.values, test_size))

2679066


In [5]:
# Bidirectional LSTM with attention
# the inputs are padded; sequence_length is used to specify the actual lengths
def bilstm_fn(features, labels, mode, params):
    attn_size = 10
    n_units = 100
    cell_dropout = 0.25
    emb_dim = 300
    vocab_size = 539953
    seq_len=tf.reshape(features["len"], [-1])
#     print(features)
    input_layer = tf.contrib.layers.embed_sequence(
        features["seq"], vocab_size, emb_dim, trainable=False,
        initializer=params["embedding_initializer"])
    print("Embedding shape:", input_layer.shape)
    
    # BiLSTM 1
    with tf.variable_scope("BiLSTM1"):
        lstm_fw_cell1 = tf.nn.rnn_cell.DropoutWrapper(tf.nn.rnn_cell.LSTMCell(n_units), cell_dropout)
        lstm_bw_cell1 = tf.nn.rnn_cell.DropoutWrapper(tf.nn.rnn_cell.LSTMCell(n_units), cell_dropout)
        (outputs_fw1, outputs_bw1), final_states1 = tf.nn.bidirectional_dynamic_rnn(
            lstm_fw_cell1, lstm_bw_cell1, input_layer, dtype=tf.float32, sequence_length=seq_len)
        outputs1 = tf.concat([outputs_fw1, outputs_bw1], axis=2)
        print("Outputs1 shape:", outputs1.shape)
    dropout1 = tf.layers.dropout(outputs1, 0.5)
               
    with tf.variable_scope("Attention"):
        e = tf.layers.dense(dropout1, 1, activation=tf.tanh)       
        print("e shape", e.shape)
        a = tf.nn.softmax(e, axis=1)
        print("a shape", a.shape)
        r = tf.reduce_sum(dropout1*a, 1)
        print("r shape", r.shape)
        
    logits = tf.layers.dense(inputs=r, units=n_classes)
    print("Logits shape:", logits.shape)
    print([v.name for v in tf.trainable_variables()])
    
    if mode == tf.estimator.ModeKeys.PREDICT:
        probs = tf.nn.softmax(logits)
        pred_indices = tf.argmax(probs, 1)
        preds = {"class":pred_indices, "probabilities":probs, "alphas":a}
        export_outputs = {"prediction":tf.estimator.export.PredictOutput(preds)}
        return tf.estimator.EstimatorSpec(mode, predictions=preds, export_outputs=export_outputs)
    
    labels = tf.one_hot(labels, n_classes)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=labels))
    l2_loss = tf.reduce_sum([tf.nn.l2_loss(v) for v in tf.trainable_variables() if "bias" not in v.name ])
    loss = loss + 0.0001*l2_loss
    tf.summary.scalar("loss", loss)
    
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.AdamOptimizer()
        train_op = optimizer.minimize(loss=loss, global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

In [144]:
# pretrained embeddings
# en_model = KeyedVectors.load_word2vec_format(r'D:\word_embedding\balanced\vectors1_w2v.txt',
#                                             binary=False, encoding="UTF-8")
# emb_dim = en_model.vector_size
# embedding_matrix = np.zeros((vocab_size, emb_dim))
# for word, i in t.word_index.items():
#     try:
#         embedding_matrix[i] = en_model.get_vector(word)
#     except KeyError:
#         continue
# def my_initializer(shape=None, dtype=tf.float32, partition_info=None):
#     assert dtype is tf.float32
#     return embedding_matrix

#### Train

In [8]:
def train_generator(): # for variable-length input
    # itertuples is much faster than iteriterms
    for row in df_data[~df_data.index.isin(test_indices)].itertuples(index=False):
        seq = [index for index in row[:-1] if index != 0]
        # sequence, sequence_length, label
        yield row[:-1], [len(seq)], [row[-1]]

def parse_record(seq, l, label):
    return {"seq":seq, "len":l}, label
    
def train_input_fn():
    dataset = tf.data.Dataset.from_generator(
        train_generator, 
        (tf.int64, tf.int64, tf.int64), 
        (tf.TensorShape([max_len]), tf.TensorShape(1), tf.TensorShape([1])))
    dataset = dataset.shuffle(batch_size*100)
    dataset = dataset.batch(batch_size).prefetch(batch_size)
    dataset = dataset.map(parse_record, num_parallel_calls=2)
    data_iter = dataset.make_one_shot_iterator()
    return data_iter.get_next()   

In [9]:
model_dir = r"F:\binary_bilstm_att_b512_df"
rmtree(model_dir, ignore_errors=True)
params = {#           'embedding_initializer': my_initializer,
              'embedding_initializer': tf.random_uniform_initializer(-1.0, 1.0)
          }
classifier = tf.estimator.Estimator(model_fn=bilstm_fn, 
                                       model_dir=model_dir, params=params)
pred = classifier.train(input_fn=train_input_fn, steps=steps)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'F:\\binary_bilstm_att_b512_df', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x00000161D570B470>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
Embedding shape: (?, 50, 300)
Outputs1 shape: (?, 50, 200)
e shape (?, 50, 1)
a shape (?, 

#### Predict

In [11]:
def test_generator():
    # itertuples is much faster than iteriterms
    for row in df_data[df_data.index.isin(test_indices)].itertuples(index=False):
        seq = [index for index in row[:-1] if index != 0]
        # yield 50 features, 1 label
        yield row[:-1], [len(seq)], [row[-1]]

def test_input_fn():
    dataset = tf.data.Dataset.from_generator(
        test_generator, 
        (tf.int64, tf.int64, tf.int64), 
        (tf.TensorShape([max_len]), tf.TensorShape(1), tf.TensorShape([1])))
    dataset = dataset.map(parse_record, num_parallel_calls=2)
    dataset = dataset.batch(batch_size*10).prefetch(batch_size*10)
    data_iter = dataset.make_one_shot_iterator()
    return data_iter.get_next()   

In [12]:
%%time
predictions = []
alphas = []
for pred in classifier.predict(input_fn=test_input_fn):
    predictions.append(pred["class"])
    alphas.append(pred["alphas"])

INFO:tensorflow:Calling model_fn.
Embedding shape: (?, 50, 300)
Outputs1 shape: (?, 50, 200)
e shape (?, 50, 1)
a shape (?, 50, 1)
r shape (?, 200)
Logits shape: (?, 2)
['BiLSTM1/bidirectional_rnn/fw/lstm_cell/kernel:0', 'BiLSTM1/bidirectional_rnn/fw/lstm_cell/bias:0', 'BiLSTM1/bidirectional_rnn/bw/lstm_cell/kernel:0', 'BiLSTM1/bidirectional_rnn/bw/lstm_cell/bias:0', 'Attention/dense/kernel:0', 'Attention/dense/bias:0', 'dense/kernel:0', 'dense/bias:0']
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from F:\binary_bilstm_att_b512_df\model.ckpt-47347
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Wall time: 4min 54s


In [13]:
true = df_data[df_data.index.isin(test_indices)]["label"]
print(f1_score(true, predictions))
print(accuracy_score(true, predictions))

0.9156826328137917
0.8495544429978492


In [14]:
df_data[df_data.index.isin(test_indices)]["label"].value_counts()/len(df_data[df_data.index.isin(test_indices)])

1    0.827766
0    0.172234
Name: label, dtype: float64

In [15]:
for line in classification_report(true, predictions).split("\n\n"):
    print(line)

              precision    recall  f1-score   support
           0       0.75      0.19      0.30    439072
           1       0.85      0.99      0.92   2110209
   micro avg       0.85      0.85      0.85   2549281
   macro avg       0.80      0.59      0.61   2549281
weighted avg       0.84      0.85      0.81   2549281



#### Visualize attention

In [16]:
# read index2word
with open(r"D:\stocktwits_text\2018 binary\tokenizer.pkl", "rb") as in_f:
    tokenizer = pickle.load(in_f)
    
index2word = {}
for word, index in tokenizer.word_index.items():
    new_word = word.replace("<", "*").replace(">", "*")
    index2word[index] = new_word
print(len(index2word))

Using TensorFlow backend.


539952


In [21]:
out_dir = r"D:\word_embedding\binary\attention visual"
file_counter = 0
html_f = None
for i, row in enumerate(df_data[df_data.index.isin(test_indices)].itertuples(index=False)):
    if i % 1000 == 0:
        if html_f:
            html_f.close()
        file_counter += 1
        if file_counter > 1:
            break
        out_file = os.path.join(out_dir, str(file_counter)+".html")
        html_f = open(out_file, "w", encoding="utf-8", newline="")
    words = [index2word[index] for index in row[:-1] if index !=0]
    html_f.write(str(row[-1]))
    html_f.write("&emsp;")
    html_f.write(str(predictions[i]))
    html_f.write("&emsp;")
    for word, alpha in zip(words, alphas[i][:len(words)]):
        html_f.write('<font style="background: rgba(255, 255, 0, %f)">%s</font>\n' % (min(alpha*2, 1), word))
    html_f.write('<br>\n')
html_f.close()

#### Some random texts (True, Predicted, Attention)

1&emsp;1&emsp;<font style="background: rgba(255, 255, 0, 0.052389)">*sym*</font>
<font style="background: rgba(255, 255, 0, 0.077649)">acelrx</font>
<font style="background: rgba(255, 255, 0, 0.089694)">remains</font>
<font style="background: rgba(255, 255, 0, 0.073007)">an</font>
<font style="background: rgba(255, 255, 0, 0.163215)">attractive</font>
<font style="background: rgba(255, 255, 0, 0.181180)">buy</font>
<font style="background: rgba(255, 255, 0, 0.052955)">after</font>
<font style="background: rgba(255, 255, 0, 0.137382)">strong</font>
<font style="background: rgba(255, 255, 0, 0.090343)">run</font>
<font style="background: rgba(255, 255, 0, 0.029486)">*sym*</font>
<font style="background: rgba(255, 255, 0, 0.029477)">*url*</font>
<br>
1&emsp;1&emsp;<font style="background: rgba(255, 255, 0, 0.059220)">*sym*</font>
<font style="background: rgba(255, 255, 0, 0.178972)">bears</font>
<font style="background: rgba(255, 255, 0, 0.031355)">are</font>
<font style="background: rgba(255, 255, 0, 0.033875)">right</font>
<font style="background: rgba(255, 255, 0, 0.039870)">we</font>
<font style="background: rgba(255, 255, 0, 0.026717)">need</font>
<font style="background: rgba(255, 255, 0, 0.072484)">to</font>
<font style="background: rgba(255, 255, 0, 0.086952)">break</font>
<font style="background: rgba(255, 255, 0, 0.105274)">0k</font>
<font style="background: rgba(255, 255, 0, 0.027058)">to</font>
<font style="background: rgba(255, 255, 0, 0.034559)">have</font>
<font style="background: rgba(255, 255, 0, 0.036138)">a</font>
<font style="background: rgba(255, 255, 0, 0.029808)">bull</font>
<font style="background: rgba(255, 255, 0, 0.031549)">rally</font>
<font style="background: rgba(255, 255, 0, 0.025186)">!</font>
<font style="background: rgba(255, 255, 0, 0.029873)">until</font>
<font style="background: rgba(255, 255, 0, 0.026196)">then</font>
<font style="background: rgba(255, 255, 0, 0.025874)">it</font>
<font style="background: rgba(255, 255, 0, 0.028296)">’</font>
<font style="background: rgba(255, 255, 0, 0.037493)">s</font>
<font style="background: rgba(255, 255, 0, 0.027772)">a</font>
<font style="background: rgba(255, 255, 0, 0.108512)">dead</font>
<font style="background: rgba(255, 255, 0, 0.028582)">cat</font>
<font style="background: rgba(255, 255, 0, 0.038041)">bounce</font>
<font style="background: rgba(255, 255, 0, 0.028546)">!</font>
<font style="background: rgba(255, 255, 0, 0.026300)">!</font>
<font style="background: rgba(255, 255, 0, 0.026128)">!</font>
<font style="background: rgba(255, 255, 0, 0.048245)">long</font>
<font style="background: rgba(255, 255, 0, 0.146204)">term</font>
<font style="background: rgba(255, 255, 0, 0.045844)">bull</font>
<br>
1&emsp;1&emsp;<font style="background: rgba(255, 255, 0, 0.035529)">"</font>
<font style="background: rgba(255, 255, 0, 0.035982)">*user*</font>
<font style="background: rgba(255, 255, 0, 0.037448)">*sym*</font>
<font style="background: rgba(255, 255, 0, 0.042898)">clever</font>
<font style="background: rgba(255, 255, 0, 0.055007)">idea</font>
<font style="background: rgba(255, 255, 0, 0.038678)">.</font>
<font style="background: rgba(255, 255, 0, 0.188346)">views</font>
<font style="background: rgba(255, 255, 0, 0.037642)">are</font>
<font style="background: rgba(255, 255, 0, 0.052121)">increasing</font>
<font style="background: rgba(255, 255, 0, 0.035293)">quickly</font>
<font style="background: rgba(255, 255, 0, 0.036610)">.</font>
<font style="background: rgba(255, 255, 0, 0.054814)">*url*</font>
<br>
1&emsp;1&emsp;<font style="background: rgba(255, 255, 0, 0.043114)">*sym*</font>
<font style="background: rgba(255, 255, 0, 0.036301)">all</font>
<font style="background: rgba(255, 255, 0, 0.034180)">the</font>
<font style="background: rgba(255, 255, 0, 0.036100)">specialists</font>
<font style="background: rgba(255, 255, 0, 0.039958)">being</font>
<font style="background: rgba(255, 255, 0, 0.121492)">proven</font>
<font style="background: rgba(255, 255, 0, 0.053464)">wrong</font>
<font style="background: rgba(255, 255, 0, 0.034729)">!</font>
<font style="background: rgba(255, 255, 0, 0.038282)">are</font>
<font style="background: rgba(255, 255, 0, 0.034995)">they</font>
<font style="background: rgba(255, 255, 0, 0.062071)">losing</font>
<font style="background: rgba(255, 255, 0, 0.034439)">their</font>
<font style="background: rgba(255, 255, 0, 0.034442)">“</font>
<font style="background: rgba(255, 255, 0, 0.034678)">jobs</font>
<font style="background: rgba(255, 255, 0, 0.034889)">”</font>
<font style="background: rgba(255, 255, 0, 0.038819)">?</font>
<font style="background: rgba(255, 255, 0, 0.040360)">lol</font>
<font style="background: rgba(255, 255, 0, 0.034939)">.</font>
<font style="background: rgba(255, 255, 0, 0.076910)">don</font>
<font style="background: rgba(255, 255, 0, 0.037010)">’</font>
<font style="background: rgba(255, 255, 0, 0.038514)">t</font>
<font style="background: rgba(255, 255, 0, 0.034359)">like</font>
<font style="background: rgba(255, 255, 0, 0.034224)">btc</font>
<font style="background: rgba(255, 255, 0, 0.040073)">?</font>
<font style="background: rgba(255, 255, 0, 0.037049)">just</font>
<font style="background: rgba(255, 255, 0, 0.069133)">don</font>
<font style="background: rgba(255, 255, 0, 0.035087)">’</font>
<font style="background: rgba(255, 255, 0, 0.036677)">t</font>
<font style="background: rgba(255, 255, 0, 0.046205)">buy</font>
<font style="background: rgba(255, 255, 0, 0.037515)">it</font>
<br>
0&emsp;1&emsp;<font style="background: rgba(255, 255, 0, 0.035360)">*user*</font>
<font style="background: rgba(255, 255, 0, 0.035963)">its</font>
<font style="background: rgba(255, 255, 0, 0.038377)">just</font>
<font style="background: rgba(255, 255, 0, 0.074419)">rumor</font>
<font style="background: rgba(255, 255, 0, 0.038514)">at</font>
<font style="background: rgba(255, 255, 0, 0.035952)">this</font>
<font style="background: rgba(255, 255, 0, 0.046910)">point</font>
<font style="background: rgba(255, 255, 0, 0.035467)">but</font>
<font style="background: rgba(255, 255, 0, 0.041881)">markets</font>
<font style="background: rgba(255, 255, 0, 0.036030)">are</font>
<font style="background: rgba(255, 255, 0, 0.045521)">efficient</font>
<font style="background: rgba(255, 255, 0, 0.040862)">and</font>
<font style="background: rgba(255, 255, 0, 0.039749)">its</font>
<font style="background: rgba(255, 255, 0, 0.056964)">trading</font>
<font style="background: rgba(255, 255, 0, 0.051026)">close</font>
<font style="background: rgba(255, 255, 0, 0.038589)">to</font>
<font style="background: rgba(255, 255, 0, 0.040803)">the</font>
<font style="background: rgba(255, 255, 0, 0.036763)">rumor</font>
<font style="background: rgba(255, 255, 0, 0.080405)">price</font>
<font style="background: rgba(255, 255, 0, 0.072367)">and</font>
<font style="background: rgba(255, 255, 0, 0.040221)">not</font>
<font style="background: rgba(255, 255, 0, 0.035984)">the</font>
<font style="background: rgba(255, 255, 0, 0.035840)">merger</font>
<font style="background: rgba(255, 255, 0, 0.038072)">price</font>
<font style="background: rgba(255, 255, 0, 0.035653)">.</font>
<br>
1&emsp;1&emsp;<font style="background: rgba(255, 255, 0, 0.039893)">*sym*</font>
<font style="background: rgba(255, 255, 0, 0.048209)">0</font>
<font style="background: rgba(255, 255, 0, 0.046562)">%</font>
<font style="background: rgba(255, 255, 0, 0.188759)">buy</font>
<font style="background: rgba(255, 255, 0, 0.069592)">signal</font>
<font style="background: rgba(255, 255, 0, 0.041054)">for</font>
<font style="background: rgba(255, 255, 0, 0.029568)">the</font>
<font style="background: rgba(255, 255, 0, 0.049102)">next</font>
<font style="background: rgba(255, 255, 0, 0.029125)">0</font>
<font style="background: rgba(255, 255, 0, 0.029760)">days</font>
<font style="background: rgba(255, 255, 0, 0.030501)">(</font>
<font style="background: rgba(255, 255, 0, 0.067463)">swaps</font>
<font style="background: rgba(255, 255, 0, 0.031395)">are</font>
<font style="background: rgba(255, 255, 0, 0.031287)">possible</font>
<font style="background: rgba(255, 255, 0, 0.032420)">before</font>
<font style="background: rgba(255, 255, 0, 0.036331)">)</font>
<font style="background: rgba(255, 255, 0, 0.031568)">.</font>
<font style="background: rgba(255, 255, 0, 0.035830)">scientific</font>
<font style="background: rgba(255, 255, 0, 0.067807)">trading</font>
<font style="background: rgba(255, 255, 0, 0.095729)">signal</font>
<font style="background: rgba(255, 255, 0, 0.035722)">by</font>
<font style="background: rgba(255, 255, 0, 0.111841)">algonell</font>
<br>
1&emsp;1&emsp;<font style="background: rgba(255, 255, 0, 0.096331)">*sym*</font>
<font style="background: rgba(255, 255, 0, 0.040231)">are</font>
<font style="background: rgba(255, 255, 0, 0.083473)">they</font>
<font style="background: rgba(255, 255, 0, 0.069777)">diluting</font>
<font style="background: rgba(255, 255, 0, 0.059896)">?</font>
<br>
1&emsp;1&emsp;<font style="background: rgba(255, 255, 0, 0.044868)">*sym*</font>
<font style="background: rgba(255, 255, 0, 0.038990)">models</font>
<font style="background: rgba(255, 255, 0, 0.037930)">ranked</font>
<font style="background: rgba(255, 255, 0, 0.039501)">as</font>
<font style="background: rgba(255, 255, 0, 0.037159)">the</font>
<font style="background: rgba(255, 255, 0, 0.076260)">top</font>
<font style="background: rgba(255, 255, 0, 0.037977)">two</font>
<font style="background: rgba(255, 255, 0, 0.037229)">on</font>
<font style="background: rgba(255, 255, 0, 0.037108)">the</font>
<font style="background: rgba(255, 255, 0, 0.038250)">list</font>
<font style="background: rgba(255, 255, 0, 0.039000)">with</font>
<font style="background: rgba(255, 255, 0, 0.037433)">over</font>
<font style="background: rgba(255, 255, 0, 0.038243)">0,0</font>
<font style="background: rgba(255, 255, 0, 0.042688)">evs</font>
<font style="background: rgba(255, 255, 0, 0.037659)">produced</font>
<font style="background: rgba(255, 255, 0, 0.045793)">,</font>
<font style="background: rgba(255, 255, 0, 0.045482)">more</font>
<font style="background: rgba(255, 255, 0, 0.037707)">than</font>
<font style="background: rgba(255, 255, 0, 0.038450)">0/0</font>
<font style="background: rgba(255, 255, 0, 0.037132)">of</font>
<font style="background: rgba(255, 255, 0, 0.037221)">the</font>
<font style="background: rgba(255, 255, 0, 0.057366)">total</font>
<font style="background: rgba(255, 255, 0, 0.047041)">ev</font>
<font style="background: rgba(255, 255, 0, 0.042054)">production</font>
<font style="background: rgba(255, 255, 0, 0.038443)">in</font>
<font style="background: rgba(255, 255, 0, 0.040902)">china</font>
<font style="background: rgba(255, 255, 0, 0.039697)">in</font>
<font style="background: rgba(255, 255, 0, 0.046670)">dec</font>
<br>
1&emsp;1&emsp;<font style="background: rgba(255, 255, 0, 0.040492)">*sym*</font>
<font style="background: rgba(255, 255, 0, 0.033570)">is</font>
<font style="background: rgba(255, 255, 0, 0.050844)">over</font>
<font style="background: rgba(255, 255, 0, 0.059295)">the</font>
<font style="background: rgba(255, 255, 0, 0.049266)">0,0</font>
<font style="background: rgba(255, 255, 0, 0.103405)">volume</font>
<font style="background: rgba(255, 255, 0, 0.042156)">marker</font>
<font style="background: rgba(255, 255, 0, 0.043308)">early</font>
<font style="background: rgba(255, 255, 0, 0.033148)">(</font>
<font style="background: rgba(255, 255, 0, 0.042626)">just</font>
<font style="background: rgba(255, 255, 0, 0.033544)">0</font>
<font style="background: rgba(255, 255, 0, 0.044189)">minutes</font>
<font style="background: rgba(255, 255, 0, 0.038495)">into</font>
<font style="background: rgba(255, 255, 0, 0.033031)">the</font>
<font style="background: rgba(255, 255, 0, 0.043031)">trading</font>
<font style="background: rgba(255, 255, 0, 0.042604)">day</font>
<font style="background: rgba(255, 255, 0, 0.033214)">!</font>
<font style="background: rgba(255, 255, 0, 0.049550)">)</font>
<font style="background: rgba(255, 255, 0, 0.138182)">yea</font>
<font style="background: rgba(255, 255, 0, 0.047420)">!</font>
<br>
1&emsp;0&emsp;<font style="background: rgba(255, 255, 0, 0.041511)">*sym*</font>
<font style="background: rgba(255, 255, 0, 0.044162)">ready</font>
<font style="background: rgba(255, 255, 0, 0.060449)">...</font>
<font style="background: rgba(255, 255, 0, 0.107091)">go</font>
<font style="background: rgba(255, 255, 0, 0.212175)">down</font>
<br>
1&emsp;1&emsp;<font style="background: rgba(255, 255, 0, 0.194707)">*sym*</font>
<font style="background: rgba(255, 255, 0, 0.068557)">crossover</font>
<br>
1&emsp;1&emsp;<font style="background: rgba(255, 255, 0, 0.038552)">*user*</font>
<font style="background: rgba(255, 255, 0, 0.063438)">im</font>
<font style="background: rgba(255, 255, 0, 0.042918)">pretty</font>
<font style="background: rgba(255, 255, 0, 0.037429)">curious</font>
<font style="background: rgba(255, 255, 0, 0.036765)">is</font>
<font style="background: rgba(255, 255, 0, 0.037034)">it</font>
<font style="background: rgba(255, 255, 0, 0.038092)">a</font>
<font style="background: rgba(255, 255, 0, 0.081125)">small</font>
<font style="background: rgba(255, 255, 0, 0.042586)">minority</font>
<font style="background: rgba(255, 255, 0, 0.036699)">here</font>
<font style="background: rgba(255, 255, 0, 0.037262)">,</font>
<font style="background: rgba(255, 255, 0, 0.053394)">or</font>
<font style="background: rgba(255, 255, 0, 0.040197)">majority</font>
<font style="background: rgba(255, 255, 0, 0.038089)">,</font>
<font style="background: rgba(255, 255, 0, 0.039881)">that</font>
<font style="background: rgba(255, 255, 0, 0.037181)">dabble</font>
<font style="background: rgba(255, 255, 0, 0.046496)">in</font>
<font style="background: rgba(255, 255, 0, 0.039012)">calls</font>
<font style="background: rgba(255, 255, 0, 0.037587)">etc</font>
<font style="background: rgba(255, 255, 0, 0.067299)">?</font>
<br>
1&emsp;1&emsp;<font style="background: rgba(255, 255, 0, 0.046875)">*sym*</font>
<font style="background: rgba(255, 255, 0, 0.103596)">0,0</font>
<font style="background: rgba(255, 255, 0, 0.185817)">block</font>
<font style="background: rgba(255, 255, 0, 0.124298)">*user*</font>
<br>
1&emsp;1&emsp;<font style="background: rgba(255, 255, 0, 0.038503)">*user*</font>
<font style="background: rgba(255, 255, 0, 0.036615)">-</font>
<font style="background: rgba(255, 255, 0, 0.037284)">thanks</font>
<font style="background: rgba(255, 255, 0, 0.037737)">for</font>
<font style="background: rgba(255, 255, 0, 0.038922)">the</font>
<font style="background: rgba(255, 255, 0, 0.042658)">alert</font>
<font style="background: rgba(255, 255, 0, 0.041492)">-</font>
<font style="background: rgba(255, 255, 0, 0.045695)">do</font>
<font style="background: rgba(255, 255, 0, 0.040977)">you</font>
<font style="background: rgba(255, 255, 0, 0.061538)">think</font>
<font style="background: rgba(255, 255, 0, 0.036852)">it</font>
<font style="background: rgba(255, 255, 0, 0.036496)">will</font>
<font style="background: rgba(255, 255, 0, 0.134976)">gap</font>
<font style="background: rgba(255, 255, 0, 0.041293)">to</font>
<font style="background: rgba(255, 255, 0, 0.036038)">.</font>
<font style="background: rgba(255, 255, 0, 0.038282)">0</font>
<font style="background: rgba(255, 255, 0, 0.064366)">today</font>
<font style="background: rgba(255, 255, 0, 0.044606)">?</font>